## Calculating lobe volumes of all CTs

In [ ]:
import os
import csv
import numpy as np
import nibabel as nib

# === Input & Output paths
input_folder = r"------ INSERT PATH HERE ------"
output_csv = r"------ INSERT PATH HERE ------"

# === Collect all top-level .nii.gz files
nii_files = [f for f in os.listdir(input_folder) if f.endswith(".nii.gz") and os.path.isfile(os.path.join(input_folder, f))]

# === Initialize CSV
with open(output_csv, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["MRN", "RUL_vol_mL", "RML_vol_mL", "RLL_vol_mL", "LUL_vol_mL", "LLL_vol_mL"])  # Header

    for fname in sorted(nii_files):
        mrn = fname.replace(".nii.gz", "")
        fpath = os.path.join(input_folder, fname)

        try:
            # Load segmentation
            img = nib.load(fpath)
            data = img.get_fdata()
            voxel_volume = np.prod(img.header.get_zooms())  # in mm³

            # Compute volumes (in mL)
            volumes = [(data == i).sum() * voxel_volume / 1000 for i in range(1, 6)]
            rounded = [round(v, 2) for v in volumes]

            writer.writerow([mrn] + rounded)
            print(f"✅ {mrn}: " + ", ".join([f"Label{i+1}={v:.1f}mL" for i, v in enumerate(volumes)]))

        except Exception as e:
            print(f"❌ Failed to process {fname}: {e}")

print(f"\n📄 Volume CSV written to: {output_csv}")
